# Population de la BDD

In [13]:
pip install pyhwmonitor influxdb-client

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable



ERROR: Could not find a version that satisfies the requirement pyhwmonitor (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for pyhwmonitor


In [ ]:
import time
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from pyhwmonitor.core import SensorMonitor

# Connexion InfluxDB
INFLUXDB_URL = "http://localhost:8086"
INFLUXDB_TOKEN = "my-secret-token"
INFLUXDB_ORG = "my-org"
INFLUXDB_BUCKET = "my-bucket"


In [12]:
client = influxdb_client.InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)

def get_windows_temperatures():
    monitor = SensorMonitor()
    monitor.scan()
    data_points = []
    
    for sensor in monitor.cpu_temperatures():
        point = influxdb_client.Point("temperature")\
            .tag("sensor", "CPU")\
            .tag("label", sensor.label)\
            .field("value", sensor.value)
        data_points.append(point)
    
    return data_points

def main():
    print("📡 Envoi des températures à InfluxDB...")
    try:
        while True:
            points = get_windows_temperatures()
            if points:
                write_api.write(bucket=INFLUXDB_BUCKET, org=INFLUXDB_ORG, record=points)
                print(f"✅ {len(points)} mesures envoyées à InfluxDB.")
            else:
                print("⚠️ Aucune température détectée.")

            time.sleep(5)
    except KeyboardInterrupt:
        print("\n⏹ Arrêt du script.")
    finally:
        client.close()

if __name__ == "__main__":
    main()

📡 Envoi des températures à InfluxDB...


NameError: name 'SensorMonitor' is not defined